# Quickstart: Agentic retrieval in Azure AI Search

Use this notebook to get started with [agentic retrieval](https://learn.microsoft.com/azure/search/search-agentic-retrieval-concept) in Azure AI Search, which integrates conversation history and large language models (LLMs) on Azure OpenAI to plan, retrieve, and synthesize complex queries.

Steps in this notebook include:

+ Creating an `earth_at_night` search index.

+ Loading the index with documents from a GitHub URL.

+ Creating an `earth-search-agent` in Azure AI Search that points to an LLM for query planning.

+ Using the agent to fetch and rank relevant information from the index.

+ Generating answers using the Azure OpenAI client.

This notebook provides a high-level demonstration of agentic retrieval. For more detailed guidance, see [Quickstart: Run agentic retrieval in Azure AI Search](https://learn.microsoft.com/azure/search/search-get-started-agentic-retrieval).

## Prerequisites

+ An [Azure AI Search service](https://learn.microsoft.com/azure/search/search-create-service-portal) on the Basic tier or higher with [semantic ranker enabled](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable).

+ An [Azure OpenAI resource](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource).

+ A [supported model](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models) deployed to your Azure OpenAI resource. This notebook uses `gpt-4o-mini`.

## Configure access

This notebook assumes authentication and authorization using Microsoft Entra ID and role assignments. It also assumes that you run the code from your local device.

To configure role-based access:

1. Sign in to the [Azure portal](https://portal.azure.com).

1. [Enable role-based access](https://learn.microsoft.com/azure/search/search-security-enable-roles) on your Azure AI Search service.

1. [Create a system-assigned managed identity](https://learn.microsoft.com/azure/search/search-howto-managed-identities-data-sources#create-a-system-managed-identity) on your Azure AI Search service.

1. On your Azure AI Search service, [assign the following roles](https://learn.microsoft.com/azure/search/search-security-rbac#how-to-assign-roles-in-the-azure-portal) to yourself.

   + **Owner/Contributor** or **Search Service Contributor**

   + **Search Index Data Contributor**

   + **Search Index Data Reader**

1. On your Azure OpenAI resource, assign **Cognitive Services User** to the managed identity of your search service.

## Set up connections

The `sample.env` file contains environment variables for connections to Azure AI Search and Azure OpenAI. Agentic retrieval requires these connections for document retrieval, query planning, query execution, and answer generation.

To set up connections:

1. Sign in to the [Azure portal](https://portal.azure.com).

2. Retrieve the endpoints for both Azure AI Search and Azure OpenAI.

3. Save the `sample.env` file as `.env` on your local device.

4. Update the `.env` file with the retrieved endpoints.

## Install packages and load connections

This step installs the packages for this notebook and establishes connections to Azure AI Search and Azure OpenAI.

In [ ]:
#r "nuget: Azure.Search.Documents, 11.7.0-beta.4"
#r "nuget: Azure.Identity, 1.14.0"
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget:dotenv.net, 3.2.1"

In [2]:
using dotenv.net;
using Azure.Identity;

// .env should be in the same directory as this notebook
DotEnv.Load(options: new DotEnvOptions(envFilePaths: new[] { ".env" }, ignoreExceptions: false));

// Get environment variables with defaults where appropriate
string answerModel = Environment.GetEnvironmentVariable("ANSWER_MODEL") ?? "gpt-4o";
string endpoint = Environment.GetEnvironmentVariable("AZURE_SEARCH_ENDPOINT") 
    ?? throw new InvalidOperationException("AZURE_SEARCH_ENDPOINT is not set.");
var credential = new DefaultAzureCredential();

string indexName = Environment.GetEnvironmentVariable("AZURE_SEARCH_INDEX") ?? "earth_at_night";
string azureOpenAiEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT") 
    ?? throw new InvalidOperationException("AZURE_OPENAI_ENDPOINT is not set.");
string azureOpenAiGptDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_GPT_DEPLOYMENT") ?? "gpt-4o";
string azureOpenAiGptModel = Environment.GetEnvironmentVariable("AZURE_OPENAI_GPT_MODEL") ?? "gpt-4o";
string azureOpenAiApiVersion = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_VERSION") ?? "2025-03-01-preview";
string azureOpenAiEmbeddingDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_EMBEDDING_DEPLOYMENT") ?? "text-embedding-3-large";
string azureOpenAiEmbeddingModel = Environment.GetEnvironmentVariable("AZURE_OPENAI_EMBEDDING_MODEL") ?? "text-embedding-3-large";
string agentName = Environment.GetEnvironmentVariable("AZURE_SEARCH_AGENT_NAME") ?? "earth-search-agent";
string apiVersion = "2025-05-01-Preview";

## Create an index in Azure AI Search

This step creates a search index that contains plain text and vector content. You can use an existing index, but it must meet the criteria for [agentic retrieval workloads](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-index). The primary schema requirement is a semantic configuration with a `default_configuration_name`.

In [3]:
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;

// Define the fields for the index
var fields = new List<SearchField>
{
    new SimpleField("id", SearchFieldDataType.String) { IsKey = true, IsFilterable = true, IsSortable = true, IsFacetable = true },
    new SearchField("page_chunk", SearchFieldDataType.String) { IsFilterable = false, IsSortable = false, IsFacetable = false },
    new SearchField("page_embedding_text_3_large", SearchFieldDataType.Collection(SearchFieldDataType.Single)) { VectorSearchDimensions = 3072, VectorSearchProfileName = "hnsw_text_3_large" },
    new SimpleField("page_number", SearchFieldDataType.Int32) { IsFilterable = true, IsSortable = true, IsFacetable = true }
};

// Define the vectorizer
var vectorizer = new AzureOpenAIVectorizer(vectorizerName: "azure_openai_text_3_large")
{
    Parameters = new AzureOpenAIVectorizerParameters
    {
        ResourceUri = new Uri(azureOpenAiEndpoint),
        DeploymentName = azureOpenAiEmbeddingDeployment,
        ModelName = azureOpenAiEmbeddingModel
    }
};

// Define the vector search profile and algorithm
var vectorSearch = new VectorSearch()
{
    Profiles =
    {
        new VectorSearchProfile(
            name: "hnsw_text_3_large",
            algorithmConfigurationName: "alg"
        )
        {
            VectorizerName = "azure_openai_text_3_large"
        }
    },
    Algorithms =
    {
        new HnswAlgorithmConfiguration(name: "alg")
    },
    Vectorizers =
    {
        vectorizer
    }
};

// Define semantic configuration
var semanticConfig = new SemanticConfiguration(
    name: "semantic_config",
    prioritizedFields: new SemanticPrioritizedFields
    {
        ContentFields = { new SemanticField("page_chunk") }
    }
);

var semanticSearch = new SemanticSearch()
{
    DefaultConfigurationName = "semantic_config",
    Configurations =
    {
        semanticConfig
    }
};

// Create the index
var index = new SearchIndex(indexName)
{
    Fields = fields,
    VectorSearch = vectorSearch,
    SemanticSearch = semanticSearch
};

// Create the index client and create or update the index
var indexClient = new SearchIndexClient(new Uri(endpoint), credential);
await indexClient.CreateOrUpdateIndexAsync(index);

Console.WriteLine($"Index '{indexName}' created or updated successfully");

Index 'earth_at_night' created or updated successfully


## Upload sample documents

This notebook uses data from NASA's Earth at Night e-book. The data is retrieved from the [azure-search-sample-data](https://github.com/Azure-Samples/azure-search-sample-data) repository on GitHub and passed to the search client for indexing.

In [4]:
using System.Net.Http;
using System.Text.Json;
using Azure.Search.Documents;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Models;

// Download the documents from the GitHub URL
string url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json";
var httpClient = new HttpClient();
var response = await httpClient.GetAsync(url);
response.EnsureSuccessStatusCode();
var json = await response.Content.ReadAsStringAsync();

var documents = JsonSerializer.Deserialize<List<Dictionary<string, object>>>(json);
var searchClient = new SearchClient(new Uri(endpoint), indexName, credential);
var searchIndexingBufferedSender = new SearchIndexingBufferedSender<Dictionary<string, object>>(
    searchClient,
    new SearchIndexingBufferedSenderOptions<Dictionary<string, object>>
    {
        KeyFieldAccessor = doc => doc["id"].ToString(),
    }
);

await searchIndexingBufferedSender.UploadDocumentsAsync(documents);
await searchIndexingBufferedSender.FlushAsync();

Console.WriteLine($"Documents uploaded to index '{indexName}'");

Documents uploaded to index 'earth_at_night'


## Create an agent in Azure AI Search

This step creates a knowledge agent, which acts as a wrapper for the LLM you deployed to Azure OpenAI. The LLM is used to send queries to an agentic retrieval pipeline.

In [1]:
using Azure.Search.Documents.Indexes.Models;

var openAiParameters = new AzureOpenAIVectorizerParameters
{
    ResourceUri = new Uri(azureOpenAiEndpoint),
    DeploymentName = azureOpenAiGptDeployment,
    ModelName = azureOpenAiGptModel
};

var agentModel = new KnowledgeAgentAzureOpenAIModel(azureOpenAIParameters: openAiParameters);

var targetIndex = new KnowledgeAgentTargetIndex(indexName)
{
    DefaultRerankerThreshold = 2.5f
};

// Create the knowledge agent
var agent = new KnowledgeAgent(
    name: agentName,
    models: new[] { agentModel },
    targetIndexes: new[] { targetIndex });
await indexClient.CreateOrUpdateKnowledgeAgentAsync(agent);
Console.WriteLine($"Knowledge agent '{agentName}' created or updated successfully");

Error: (1,7): error CS0246: The type or namespace name 'Azure' could not be found (are you missing a using directive or an assembly reference?)
(3,28): error CS0246: The type or namespace name 'AzureOpenAIVectorizerParameters' could not be found (are you missing a using directive or an assembly reference?)
(5,27): error CS0103: The name 'azureOpenAiEndpoint' does not exist in the current context
(6,22): error CS0103: The name 'azureOpenAiGptDeployment' does not exist in the current context
(7,17): error CS0103: The name 'azureOpenAiGptModel' does not exist in the current context
(10,22): error CS0246: The type or namespace name 'KnowledgeAgentAzureOpenAIModel' could not be found (are you missing a using directive or an assembly reference?)
(12,23): error CS0246: The type or namespace name 'KnowledgeAgentTargetIndex' could not be found (are you missing a using directive or an assembly reference?)
(12,49): error CS0103: The name 'indexName' does not exist in the current context
(

## Set up messages

Messages are the input for the retrieval route and contain the conversation history. Each message includes a `role` that indicates its origin, such as `assistant` or `user`, and `content` in natural language. The LLM you use determines which roles are valid.

In [6]:
string instructions = @"
A Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with ""I don't know"".
";

var messages = new List<Dictionary<string, string>>
{
    new Dictionary<string, string>
    {
        { "role", "system" },
        { "content", instructions }
    }
};

## Use agentic retrieval to fetch results

This step runs the retrieval pipeline to extract relevant information from your search index. Based on the messages and parameters on the retrieval request, the LLM:

1. Analyzes the entire conversation history to determine the underlying information need.

1. Breaks down the compound user query into focused subqueries.
 
1. Runs each subquery simultaneously against text fields and vector embeddings in your index.

1. Uses semantic ranker to rerank the results of all subqueries.

1. Merges the results into a single string.

In [7]:
using Azure.Search.Documents.Agents;
using Azure.Search.Documents.Agents.Models;

var agentClient = new KnowledgeAgentRetrievalClient(
    endpoint: new Uri(endpoint),
    agentName: agentName,
    tokenCredential: new DefaultAzureCredential()
);

messages.Add(new Dictionary<string, string>
{
    { "role", "user" },
    { "content", @"
Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
" }
});

var retrievalResult = await agentClient.RetrieveAsync(
    retrievalRequest: new KnowledgeAgentRetrievalRequest(
            messages: messages
                .Where(message => message["role"] != "system")
                .Select(
                message => new KnowledgeAgentMessage(
                    role: message["role"],
                    content: new[] { new KnowledgeAgentMessageTextContent(message["content"]) }))
                .ToList()
            )
        {
            TargetIndexParams = { new KnowledgeAgentIndexParams { IndexName = indexName, RerankerThreshold = 2.5f } }
        }
    );

messages.Add(new Dictionary<string, string>
{
    { "role", "assistant" },
    { "content", (retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text }
});

### Review the retrieval response, activity, and results

Each retrieval response from Azure AI Search includes:

+ A unified string that represents grounding data from the search results.

+ The query plan.

+ Reference data that shows which chunks of the source documents contributed to the unified string.

In [8]:
(retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text 

[{"ref_id":0,"content":"<!-- PageHeader=\"Urban Structure\" -->\n\n### Location of Phoenix, Arizona\n\nThe image depicts a globe highlighting the location of Phoenix, Arizona, in the southwestern United States, marked with a blue pinpoint on the map of North America. Phoenix is situated in the central part of Arizona, which is in the southwestern region of the United States.\n\n---\n\n### Grid of City Blocks-Phoenix, Arizona\n\nLike many large urban areas of the central and western United States, the Phoenix metropolitan area is laid out along a regular grid of city blocks and streets. While visible during the day, this grid is most evident at night, when the pattern of street lighting is clearly visible from the low-Earth-orbit vantage point of the ISS.\n\nThis astronaut photograph, taken on March 16, 2013, includes parts of several cities in the metropolitan area, including Phoenix (image right), Glendale (center), and Peoria (left). While the major street grid is oriented north-sout

In [9]:
Console.WriteLine("Activities:");
foreach (var activity in retrievalResult.Value.Activity)
{
    Console.WriteLine($"Activity Type: {activity.GetType().Name}");
    string json = JsonSerializer.Serialize(
        activity,
        activity.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

Console.WriteLine("Results");
foreach (var reference in retrievalResult.Value.References)
{
    Console.WriteLine($"Reference Type: {reference.GetType().Name}");
    string json = JsonSerializer.Serialize(
        reference,
        reference.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

Activities:
Activity Type: KnowledgeAgentModelQueryPlanningActivityRecord
{
  "InputTokens": 1262,
  "OutputTokens": 417,
  "ElapsedMs": null,
  "Id": 0
}
Activity Type: KnowledgeAgentSearchActivityRecord
{
  "TargetIndex": "earth_at_night",
  "Query": {
    "Search": "Why do suburban belts experience more December brightening than urban cores?",
    "Filter": null
  },
  "QueryTime": "2025-05-14T03:55:13.414+00:00",
  "Count": 0,
  "ElapsedMs": 743,
  "Id": 1
}
Activity Type: KnowledgeAgentSearchActivityRecord
{
  "TargetIndex": "earth_at_night",
  "Query": {
    "Search": "Why is the Phoenix nighttime street grid more visible from space than interstate stretches between midwestern cities?",
    "Filter": null
  },
  "QueryTime": "2025-05-14T03:55:13.871+00:00",
  "Count": 2,
  "ElapsedMs": 445,
  "Id": 2
}
Activity Type: KnowledgeAgentSemanticRankerActivityRecord
{
  "InputTokens": 52841,
  "ElapsedMs": null,
  "Id": 3
}
Results
Reference Type: KnowledgeAgentAzureSearchDocReference
{

## Create the Azure OpenAI client

So far, this notebook has used agentic retrieval for answer *extraction*, which you can extend to answer *generation* by using the Azure OpenAI client. This enables more detailed, context-rich responses that aren't strictly tied to indexed content.

In [10]:
using Azure.AI.OpenAI;
using OpenAI.Chat;

AzureOpenAIClient azureClient = new(
    new Uri(azureOpenAiEndpoint),
    new DefaultAzureCredential());
ChatClient chatClient = azureClient.GetChatClient(azureOpenAiGptDeployment);

### Use the Chat Completions API to generate an answer

One option for answer generation is the Chat Completions API, which passes the conversation history to the LLM for processing.

In [11]:
List<ChatMessage> chatMessages = messages
    .Select<Dictionary<string, string>, ChatMessage>(m => m["role"] switch
    {
        "user" => new UserChatMessage(m["content"]),
        "assistant" => new AssistantChatMessage(m["content"]),
        "system" => new SystemChatMessage(m["content"]),
        _ => null
    })
    .Where(m => m != null)
    .ToList();


var result = await chatClient.CompleteChatAsync(chatMessages);

Console.WriteLine($"[ASSISTANT]: {result.Value.Content[0].Text.Replace(".", "\n")}");




[ASSISTANT]: Suburban belts display larger December brightening than urban cores because suburban areas tend to have more holiday lighting and decorations, which significantly increase visible brightness during the holiday season
 In urban cores, where light levels are already high due to existing infrastructure and city lights, the addition of holiday lighting might not have as noticeable an impact relative to existing light intensity [ref_id:0]


The Phoenix nighttime street grid is sharply visible from space because its metropolitan area is laid out in a regular grid pattern, which is emphasized by the street lighting
 Major intersections, commercial properties, and other brightly lit areas contribute to this visibility
 In contrast, large stretches of interstate between midwestern cities do not have such concentrated sources of lighting and are often less populated, resulting in comparatively dim appearances from space [ref_id:1]



## Continue the conversation

This step continues the conversation with the knowledge agent, building upon the previous messages and queries to retrieve relevant information from your search index.

In [12]:
messages.Add(new Dictionary<string, string>
{
    { "role", "user" },
    { "content", "How do I find lava at night?" }
});


var retrievalResult = await agentClient.RetrieveAsync(
    retrievalRequest: new KnowledgeAgentRetrievalRequest(
            messages: messages
                .Where(message => message["role"] != "system")
                .Select(
                message => new KnowledgeAgentMessage(
                    role: message["role"],
                    content: new[] { new KnowledgeAgentMessageTextContent(message["content"]) }))
                .ToList()
            )
        {
            TargetIndexParams = { new KnowledgeAgentIndexParams { IndexName = indexName, RerankerThreshold = 2.5f } }
        }
    );

messages.Add(new Dictionary<string, string>
{
    { "role", "assistant" },
    { "content", (retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text }
});


### Review the retrieval response, activity, and results

In [13]:
(retrievalResult.Value.Response[0].Content[0] as KnowledgeAgentMessageTextContent).Text 

[{"ref_id":0,"content":"<!-- PageHeader=\"Volcanoes\" -->\n\n### Nighttime Glow at Mount Etna - Italy\n\nAt about 2:30 a.m. local time on March 16, 2017, the VIIRS DNB on the Suomi NPP satellite captured this nighttime image of lava flowing on Mount Etna in Sicily, Italy. Etna is one of the world's most active volcanoes.\n\n#### Figure: Location of Mount Etna\nA world globe is depicted, with a marker indicating the location of Mount Etna in Sicily, Italy, in southern Europe near the center of the Mediterranean Sea.\n\n<!-- PageFooter=\"Earth at Night\" -->\n<!-- PageNumber=\"48\" -->"},{"ref_id":1,"content":"## Nature's Light Shows\n\nAt night, with the light of the Sun removed, nature's brilliant glow from Earth's surface becomes visible to the naked eye from space. Some of Earth's most spectacular light shows are natural, like the aurora borealis, or Northern Lights, in the Northern Hemisphere (aurora australis, or Southern Lights, in the Southern Hemisphere). The auroras are natural

In [14]:
Console.WriteLine("Activities:");
foreach (var activity in retrievalResult.Value.Activity)
{
    Console.WriteLine($"Activity Type: {activity.GetType().Name}");
    string json = JsonSerializer.Serialize(
        activity,
        activity.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

Console.WriteLine("Results");
foreach (var reference in retrievalResult.Value.References)
{
    Console.WriteLine($"Reference Type: {reference.GetType().Name}");
    string json = JsonSerializer.Serialize(
        reference,
        reference.GetType(),
        new JsonSerializerOptions { WriteIndented = true }
    );
    Console.WriteLine(json);
}

Activities:
Activity Type: KnowledgeAgentModelQueryPlanningActivityRecord
{
  "InputTokens": 2138,
  "OutputTokens": 316,
  "ElapsedMs": null,
  "Id": 0
}
Activity Type: KnowledgeAgentSearchActivityRecord
{
  "TargetIndex": "earth_at_night",
  "Query": {
    "Search": "methods to find lava flows at night",
    "Filter": null
  },
  "QueryTime": "2025-05-14T03:55:25.211+00:00",
  "Count": 4,
  "ElapsedMs": 393,
  "Id": 1
}
Activity Type: KnowledgeAgentSearchActivityRecord
{
  "TargetIndex": "earth_at_night",
  "Query": {
    "Search": "equipment for detecting lava at night",
    "Filter": null
  },
  "QueryTime": "2025-05-14T03:55:25.548+00:00",
  "Count": 0,
  "ElapsedMs": 324,
  "Id": 2
}
Activity Type: KnowledgeAgentSearchActivityRecord
{
  "TargetIndex": "earth_at_night",
  "Query": {
    "Search": "safety tips for observing lava at night",
    "Filter": null
  },
  "QueryTime": "2025-05-14T03:55:25.865+00:00",
  "Count": 0,
  "ElapsedMs": 317,
  "Id": 3
}
Activity Type: KnowledgeAg

## Generate answer

In [15]:
List<ChatMessage> chatMessages = messages
    .Select<Dictionary<string, string>, ChatMessage>(m => m["role"] switch
    {
        "user" => new UserChatMessage(m["content"]),
        "assistant" => new AssistantChatMessage(m["content"]),
        "system" => new SystemChatMessage(m["content"]),
        _ => null
    })
    .Where(m => m != null)
    .ToList();


var result = await chatClient.CompleteChatAsync(chatMessages);

Console.WriteLine($"[ASSISTANT]: {result.Value.Content[0].Text.Replace(".", "\n")}");




[ASSISTANT]: To find lava at night, one effective method is using satellite technology that detects infrared or infrared heat signatures
 For instance, the VIIRS Day/Night Band (DNB) on satellites can detect glowing lava flows by capturing natural nightlight sources, such as moonlight, which highlight volcanic activity
 Additionally, thermal imaging satellites like Landsat can capture detailed images of lava flows using thermal, shortwave infrared, and near-infrared wavelengths, providing visibility of hot and cooling lava flows even through obstructions like clouds
 This data can then be analyzed to pinpoint active lava locations at night, making it both possible and practical for scientists and emergency responders to monitor volcanic activity remotely ([ref_id: 0, 1, 2, 3])



## Clean up objects and resources

If you no longer need Azure AI Search or Azure OpenAI, delete them from your Azure subscription. You can also start over by deleting individual objects.

### Delete the knowledge agent

In [ ]:
await indexClient.DeleteKnowledgeAgentAsync(agentName);
System.Console.WriteLine($"Knowledge agent '{agentName}' deleted successfully");

Search agent 'earth-search-agent' deleted successfully


### Delete the search index

In [17]:
await indexClient.DeleteIndexAsync(indexName);
System.Console.WriteLine($"Index '{indexName}' deleted successfully");

Index 'earth_at_night' deleted successfully
